In [75]:
!pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import igraph as ig
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn import Parameter
import torch.optim as optim
import math
import time

In [2]:
data_list = []
flist = []
def import_node_db():
  global flist,data_list
  data_list = ["Stockholm","Saarbrucken","Freiburg"]
  floor_list1 = ["floor4_1","floor4_2","floor4_3","floor4_4","floor4_5","floor4_6","floor4_7","floor4_8",
                   "floor5_1","floor5_2","floor5_3","floor5_4","floor5_5","floor5_6","floor5_7","floor5_8",
                   "floor6_1","floor6_2","floor6_3","floor6_4","floor6_5","floor6_6","floor6_7","floor6_8","floor6_9",
                   "floor6_10","floor6_11","floor6_12","floor6_13","floor6_14","floor6_15","floor6_16","floor6_17","floor6_18",
                   "floor7_1","floor7_2","floor7_3","floor7_4","floor7_5","floor7_6","floor7_7","floor7_8"]
  floor_list2 = ["seq1_cloudy1","seq1_cloudy2","seq1_cloudy3","seq1_night1","seq1_night2","seq1_night3",
                 "seq2_cloudy1","seq2_cloudy2","seq2_cloudy3","seq2_night1","seq2_night2","seq2_night3",
                 "seq3_cloudy1","seq3_cloudy2","seq3_cloudy3","seq3_cloudy4","seq3_cloudy5","seq3_night1","seq3_night2","seq3_night3","seq3_sunny1","seq3_sunny2","seq3_sunny3",
                 "seq4_cloudy1","seq4_cloudy2","seq4_cloudy3","seq4_night1","seq4_night2","seq4_night3","seq4_sunny1","seq4_sunny2","seq4_sunny3"]

  floor_list3 = ["seq1_cloudy1","seq1_cloudy2","seq1_cloudy3","seq1_night1","seq1_night2","seq1_night3","seq1_sunny1","seq1_sunny2","seq1_sunny3","seq1_sunny4",
                 "seq2_cloudy1","seq2_cloudy2","seq2_cloudy3","seq2_night1","seq2_night2","seq2_night3","seq2_sunny1","seq2_sunny2","seq2_sunny3","seq2_sunny4",
                 "seq3_cloudy1","seq3_cloudy2","seq3_cloudy3","seq3_sunny1","seq3_sunny2","seq3_sunny3"]    

  flist = [floor_list1,floor_list2,floor_list3]
  # return flist
  node_df_list = []
  for i in range(3):
    df_list = []
    for j in flist[i]:
      if(i==0):
        p=6
      else:
        p=4
      node_df1 = pd.read_table("/content/drive/MyDrive/Minor_Project/{}/{}/nodes.dat".format(data_list[i],j),sep="\s+")
      # node_df1['floor']=j
      # node_df1['loc']=data_list[i]
      for k in range(8):
        node_df1["new_neighbor_ID_{}".format(k)] = [str(col) +"-"+ str(j) + "-" + str(data_list[i][:3])  for col in node_df1["neighbor_ID_{}".format(k)]]
      # node_df1['locfloor']=data_list[i]+"-"+j
      node_df1['node_id']=[str(col) +"-"+ str(j) + "-" + str(data_list[i][:3]) for col in node_df1['node_id']]
      df_list.append(node_df1)
    node_df_list.append(pd.concat(df_list,ignore_index=True))
  main_list = pd.concat(node_df_list,ignore_index=True)
  return main_list
data = import_node_db()
data

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Minor_Project/Stockholm/floor4_1/nodes.dat'

In [78]:
node_number = 1
for i in data["node_id"]:
  # print(i)
  data = data.replace(i,node_number)
  node_number = node_number + 1

In [79]:
data.head()

,node_id,placeholder,x,y,d1,d2,d3,d4,label,d5,...,affordance_7,view_num_7,new_neighbor_ID_0,new_neighbor_ID_1,new_neighbor_ID_2,new_neighbor_ID_3,new_neighbor_ID_4,new_neighbor_ID_5,new_neighbor_ID_6,new_neighbor_ID_7
0,1,0,-1.000001,-0.200001,-1.025785,-0.097851,-1.000001,-0.200001,EV,1.205846e+09,...,-1.0,0,2,3,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto
1,2,0,-0.140001,0.259999,-0.303640,-0.002786,-0.200001,0.599999,EV,1.205846e+09,...,-1.0,0,1,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto
2,3,0,-2.020001,-0.080001,-2.015833,-0.073241,-1.800001,-0.200001,DW,1.205846e+09,...,-1.0,1,1,4,5,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto
3,4,0,-2.860001,0.499999,-3.019827,0.500790,-2.600001,0.599999,CR,1.205846e+09,...,-1.0,0,3,5,7,10,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto
4,5,0,-2.820001,-0.500001,-2.986465,-0.450972,-2.600001,-0.200001,CR,1.205846e+09,...,-1.0,1,3,4,6,7,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto


In [80]:
for i in data_list:
  for j in flist:
    for k in j:
      data.replace(str(-1)+"-"+k+"-"+i[:3],0,inplace=True)

In [81]:
data.head(20)

,node_id,placeholder,x,y,d1,d2,d3,d4,label,d5,...,affordance_7,view_num_7,new_neighbor_ID_0,new_neighbor_ID_1,new_neighbor_ID_2,new_neighbor_ID_3,new_neighbor_ID_4,new_neighbor_ID_5,new_neighbor_ID_6,new_neighbor_ID_7
0,1,0,-1.000001,-0.200001,-1.025785,-0.097851,-1.000001,-0.200001,EV,1.205846e+09,...,-1.0,0,2,3,0,0,0,0,0,0
1,2,0,-0.140001,0.259999,-0.303640,-0.002786,-0.200001,0.599999,EV,1.205846e+09,...,-1.0,0,1,0,0,0,0,0,0,0
2,3,0,-2.020001,-0.080001,-2.015833,-0.073241,-1.800001,-0.200001,DW,1.205846e+09,...,-1.0,1,1,4,5,0,0,0,0,0
3,4,0,-2.860001,0.499999,-3.019827,0.500790,-2.600001,0.599999,CR,1.205846e+09,...,-1.0,0,3,5,7,10,0,0,0,0
4,5,0,-2.820001,-0.500001,-2.986465,-0.450972,-2.600001,-0.200001,CR,1.205846e+09,...,-1.0,1,3,4,6,7,0,0,0,0
5,6,0,-2.560001,-1.300001,-2.877229,-1.491423,-2.600001,-1.000001,CR,1.205846e+09,...,-1.0,0,3,5,8,9,0,0,0,0
6,7,0,-3.700001,0.759999,-3.028493,0.786833,-3.400001,0.599999,CR,1.205846e+09,...,-1.0,1,5,4,10,86,0,0,0,0
7,8,0,-1.840001,-2.100001,-1.837850,-2.109282,-1.800001,-1.800001,PT,1.205846e+09,...,-1.0,1,6,89,109,110,0,0,0,0
8,9,0,-3.160001,-2.100001,-3.209764,-2.063027,-3.400001,-1.800001,PT,1.205846e+09,...,-1.0,0,6,87,88,89,0,0,0,0
9,10,0,-3.100001,1.579999,-3.043470,1.572917,-3.400001,1.399999,CR,1.205846e+09,...,-1.0,0,4,7,11,12,0,0,0,0


In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   node_id            12009 non-null  int64  
 1   placeholder        12009 non-null  int64  
 2   x                  12009 non-null  float64
 3   y                  12009 non-null  float64
 4   d1                 12009 non-null  float64
 5   d2                 12009 non-null  float64
 6   d3                 12009 non-null  float64
 7   d4                 12009 non-null  float64
 8   label              12009 non-null  object 
 9   d5                 12009 non-null  float64
 10  neighbor_ID_0      12009 non-null  int64  
 11  affordance_0       12009 non-null  float64
 12  view_num_0         12009 non-null  int64  
 13  neighbor_ID_1      12009 non-null  int64  
 14  affordance_1       12009 non-null  float64
 15  view_num_1         12009 non-null  int64  
 16  neighbor_ID_2      120

In [101]:
df = data.drop(['d1','d2','d3','d4','d5'], axis=1)
label_list = data['label']
label_list

0        EV
1        EV
2        DW
3        CR
4        CR
         ..
12004    BA
12005    BA
12006    BA
12007    BA
12008    BA
Name: label, Length: 12009, dtype: object

In [84]:
def get_edge(data):
  edges = []
  if(data.new_neighbor_ID_0!=0):
    edges.append([data.node_id,data.new_neighbor_ID_0])
  else:
    pass
  if(data.new_neighbor_ID_1!=0):
    edges.append([data.node_id,data.new_neighbor_ID_1])
  else:
    pass
  if(data.new_neighbor_ID_2!=0):
    edges.append([data.node_id,data.new_neighbor_ID_2])
  else:
    pass
  if(data.new_neighbor_ID_3!=0):
    edges.append([data.node_id,data.new_neighbor_ID_3])
  else:
    pass
  if(data.new_neighbor_ID_4!=0):
    edges.append([data.node_id,data.new_neighbor_ID_4])
  else:
    pass
  if(data.new_neighbor_ID_5!=0):
    edges.append([data.node_id,data.new_neighbor_ID_5])
  else:
    pass
  if(data.new_neighbor_ID_6!=0):
    edges.append([data.node_id,data.new_neighbor_ID_6])
  else:
    pass
  if(data.new_neighbor_ID_7!=0):
    edges.append([data.node_id,data.new_neighbor_ID_7])
  else:
    pass
  return edges

In [85]:
lst = []
for i in range(len(df)):
  lst.extend(get_edge(df.iloc[i,:]))
lst

[[1, 2],
 [1, 3],
 [2, 1],
 [3, 1],
 [3, 4],
 [3, 5],
 [4, 3],
 [4, 5],
 [4, 7],
 [4, 10],
 [5, 3],
 [5, 4],
 [5, 6],
 [5, 7],
 [6, 3],
 [6, 5],
 [6, 8],
 [6, 9],
 [7, 5],
 [7, 4],
 [7, 10],
 [7, 86],
 [8, 6],
 [8, 89],
 [8, 109],
 [8, 110],
 [9, 6],
 [9, 87],
 [9, 88],
 [9, 89],
 [10, 4],
 [10, 7],
 [10, 11],
 [10, 12],
 [11, 10],
 [11, 12],
 [11, 13],
 [12, 10],
 [12, 11],
 [12, 13],
 [13, 12],
 [13, 11],
 [13, 14],
 [13, 15],
 [13, 16],
 [14, 13],
 [14, 15],
 [14, 17],
 [14, 80],
 [15, 13],
 [15, 14],
 [15, 16],
 [15, 17],
 [15, 18],
 [16, 13],
 [16, 15],
 [16, 17],
 [16, 18],
 [16, 19],
 [17, 15],
 [17, 18],
 [17, 14],
 [17, 41],
 [18, 15],
 [18, 16],
 [18, 19],
 [18, 17],
 [18, 41],
 [19, 16],
 [19, 18],
 [19, 20],
 [19, 21],
 [20, 19],
 [20, 21],
 [20, 22],
 [20, 23],
 [20, 24],
 [21, 19],
 [21, 20],
 [21, 24],
 [21, 25],
 [22, 20],
 [23, 20],
 [23, 21],
 [23, 24],
 [23, 26],
 [24, 20],
 [24, 21],
 [24, 23],
 [24, 25],
 [24, 26],
 [24, 27],
 [25, 21],
 [25, 24],
 [25, 27],
 [26, 

In [86]:
adj = np.zeros(shape=(len(df),len(df)))
adj.shape

(12009, 12009)

In [87]:
for i,j in lst:
  adj[i-1,j-1] = 1
adj

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [88]:
!pip install torch
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   node_id            12009 non-null  int64  
 1   placeholder        12009 non-null  int64  
 2   x                  12009 non-null  float64
 3   y                  12009 non-null  float64
 4   label              12009 non-null  object 
 5   neighbor_ID_0      12009 non-null  int64  
 6   affordance_0       12009 non-null  float64
 7   view_num_0         12009 non-null  int64  
 8   neighbor_ID_1      12009 non-null  int64  
 9   affordance_1       12009 non-null  float64
 10  view_num_1         12009 non-null  int64  
 11  neighbor_ID_2      12009 non-null  int64  
 12  affordance_2       12009 non-null  float64
 13  view_num_2         12009 non-null  int64  
 14  neighbor_ID_3      12009 non-null  int64  
 15  affordance_3       12009 non-null  float64
 16  view_num_3         120

In [91]:
df = data.drop(['node_id','d1','d2','d3','d4','d5','placeholder','neighbor_ID_0','neighbor_ID_1','neighbor_ID_2','neighbor_ID_3',
                'neighbor_ID_4','neighbor_ID_5','neighbor_ID_6','neighbor_ID_7'], axis=1)

In [92]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [93]:
labels = encode_onehot(np.array(df['label']))
df = df.drop(['label'],axis=1)

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   x                  12009 non-null  float64
 1   y                  12009 non-null  float64
 2   affordance_0       12009 non-null  float64
 3   view_num_0         12009 non-null  int64  
 4   affordance_1       12009 non-null  float64
 5   view_num_1         12009 non-null  int64  
 6   affordance_2       12009 non-null  float64
 7   view_num_2         12009 non-null  int64  
 8   affordance_3       12009 non-null  float64
 9   view_num_3         12009 non-null  int64  
 10  affordance_4       12009 non-null  float64
 11  view_num_4         12009 non-null  int64  
 12  affordance_5       12009 non-null  float64
 13  view_num_5         12009 non-null  int64  
 14  affordance_6       12009 non-null  float64
 15  view_num_6         12009 non-null  int64  
 16  affordance_7       120

In [95]:
# import numpy as np
# import torch
# import torch.nn.functional as F
# from torch.nn import Linear
# from torch_geometric.nn import GCNConv
# from torch_geometric.data import Data

# # Define the adjacency matrix and feature array
# adj_matrix = np.random.rand(120, 120)  # Replace with your actual adjacency matrix
# feature_array = np.random.rand(120, 15)  # Replace with your actual feature array

# # Convert the adjacency matrix and feature array into a PyTorch tensor
# edge_index = torch.tensor(np.nonzero(adj_matrix), dtype=torch.long)
# x = torch.tensor(feature_array, dtype=torch.float)

# # Define the target labels
# y = torch.randint(low=0, high=2, size=(120,), dtype=torch.long)

# # Create a PyTorch Geometric Data object
# data = Data(x=x, edge_index=edge_index, y=y)

# # Define the GCN model
# class GCN(torch.nn.Module):
#     def __init__(self, hidden_channels):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(data.num_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, data.num_classes)
    
#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
#         return F.log_softmax(x, dim=1)

# # Initialize the model and define the optimizer
# model = GCN(hidden_channels=16)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# # Split the data into train, validation, and test sets
# train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
# train_mask[:80] = True
# val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
# val_mask[80:100] = True
# test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
# test_mask[100:] = True
# data.train_mask = train_mask
# data.val_mask = val_mask
# data.test_mask = test_mask

# # Train the model
# model.train()
# for epoch in range(200):
#     optimizer.zero_grad()
#     out = model(x, edge_index)
#     loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
#     loss.backward()
#     optimizer.step()

#     # Evaluate the model on the validation set
#     model.eval()
#     out = model(x, edge_index)
#     _, pred = out.max(dim=1)
#     correct = float(pred[data.val_mask].eq(data.y[data.val_mask]).sum().item())
#     val_acc = correct / data.val_mask.sum().item()
#     print('Epoch: {:03d}, Loss: {:.4f}, Validation Acc: {:.4f}'.format(epoch, loss.item(), val_acc))

# # Evaluate the model on the test set
# model.eval()
# out = model(x, edge_index)
# _, pred = out.max(dim=1)
# correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
# acc = correct / data.test_mask.sum().item()
# print('Test Accuracy: {:.4f}'.format(acc))


In [96]:
adj_matrix = adj  # Replace with your actual adjacency matrix
feature_array = np.array(df)

In [97]:
edge_index = torch.tensor(np.nonzero(adj_matrix), dtype=torch.long)
x = torch.tensor(feature_array, dtype=torch.float)

In [108]:
# Create a PyTorch Geometric Data object
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
demo_y = label_encoder.fit_transform(label_list)

y = torch.tensor(demo_y,dtype=torch.long)
data = Data(x=x, edge_index=edge_index, y=labels)

In [144]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(26, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 20)
        self.conv3 = GCNConv(20, 22)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

In [145]:
model = GCN(hidden_channels=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [146]:
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:9000] = True
val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
val_mask[9000:11000] = True
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask[11000:] = True
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [147]:
model.train()

GCN(
  (conv1): GCNConv(26, 10)
  (conv2): GCNConv(10, 20)
  (conv3): GCNConv(20, 22)
)

In [149]:
for epoch in range(300):
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = F.nll_loss(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    out = model(x, edge_index)
    _, pred = out.max(dim=1)
    correct = float(pred[val_mask].eq(y[val_mask]).sum().item())
    val_acc = correct / val_mask.sum().item()
    print('Epoch: {:03d}, Loss: {:.4f}, Validation Acc: {:.4f}'.format(epoch, loss.item(), val_acc))

Epoch: 000, Loss: 6.4361, Validation Acc: 0.2380
Epoch: 001, Loss: 6.2821, Validation Acc: 0.2430
Epoch: 002, Loss: 6.1380, Validation Acc: 0.2445
Epoch: 003, Loss: 5.9949, Validation Acc: 0.2455
Epoch: 004, Loss: 5.8524, Validation Acc: 0.2465
Epoch: 005, Loss: 5.7111, Validation Acc: 0.2435
Epoch: 006, Loss: 5.5770, Validation Acc: 0.2370
Epoch: 007, Loss: 5.4496, Validation Acc: 0.2330
Epoch: 008, Loss: 5.3289, Validation Acc: 0.2335
Epoch: 009, Loss: 5.2109, Validation Acc: 0.2325
Epoch: 010, Loss: 5.0961, Validation Acc: 0.2340
Epoch: 011, Loss: 4.9830, Validation Acc: 0.2325
Epoch: 012, Loss: 4.8727, Validation Acc: 0.2425
Epoch: 013, Loss: 4.7653, Validation Acc: 0.2490
Epoch: 014, Loss: 4.6594, Validation Acc: 0.2555
Epoch: 015, Loss: 4.5583, Validation Acc: 0.2640
Epoch: 016, Loss: 4.4622, Validation Acc: 0.2690
Epoch: 017, Loss: 4.3752, Validation Acc: 0.2735
Epoch: 018, Loss: 4.2925, Validation Acc: 0.2805
Epoch: 019, Loss: 4.2133, Validation Acc: 0.2795
Epoch: 020, Loss: 4.

In [150]:
model.eval()
out = model(x, edge_index)
_, pred = out.max(dim=1)

correct = float(pred[test_mask].eq(y[test_mask]).sum().item())
acc = correct / test_mask.sum().item()
print('Test Accuracy: {:.4f}'.format(acc))

Test Accuracy: 0.3211
